In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fmin_cg

# Define functions

In [ ]:
def featureNormalize(X):
    X_norm = X
    mu = X.mean(axis = 0)
    sigma = X.std(axis = 0)
    c = X.shape[1]
    for i in range(c):
        X_norm[:, i] = (X[:, i] - mu[i]) / sigma[i]
    return X_norm, mu, sigma

In [ ]:
def rand_initialize_weights(architecture_layers_size):
    '''
    random initialize weights using a uniform distribution
    
    param: 
        architecture_layers_size: number of neurons of each NN layer 
    '''
    list_initial_theta = []
    np.random.seed(1)
    for i in range(len(architecture_layers_size) - 1):
        list_initial_theta.insert(i, np.random.rand(architecture_layers_size[i + 1], architecture_layers_size[i] + 1))
        print(list_initial_theta[i].shape)
    return list_initial_theta

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
def sigmoid_gradient(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [ ]:
def cost_function(nn_params, X, y, architecture_layers_size, lam):
    m = X.shape[0]
    print(nn_params.shape)
    
    list_nn_params = []
    first = 0
    for i in range(len(architecture_layers_size) - 1):
        dim_r = architecture_layers_size[i+1]
        dim_c = architecture_layers_size[i] + 1
        # print('dim r:', dim_r)
        # print('dim c:', dim_c)
        # print('first:', first)
        last = first + (dim_r * dim_c)
        # print('last:', last-1)
        p = nn_params[first:last].reshape((dim_r, dim_c))
        # print(p.shape)
        list_nn_params.append(p)
        first = last
        # print((nn_params[first:(dim_r * dim_c)].reshape((dim_r, dim_c))).shape)
    
    # Forward propagation
    list_activation = []
    list_z = []
#     list_activation.insert(0, np.concatenate((np.ones((m, 1)), X), axis = 1))
    list_activation.insert(0, np.hstack((np.ones((m, 1)), X)))
    # print('a1:', list_activation[0].shape)
    if len(architecture_layers_size) > 2:
        for i in range(1, len(architecture_layers_size) - 1):
            # print('Theta:', list_nn_params[i - 1].shape)
#             list_z.insert(i - 1, np.dot(list_activation[i - 1], np.transpose(list_nn_params[i - 1]))) # matrix multiplication (righe x colonne)
            list_z.insert(i - 1, np.matmul(list_activation[i - 1], list_nn_params[i - 1].T))
            # print('z:', list_z[i - 1].shape)
#             list_activation.insert(i, np.concatenate((np.ones((m, 1)), sigmoid(list_z[i - 1])), axis = 1))
            list_activation.insert(i, np.hstack((np.ones((m, 1)), sigmoid(list_z[i - 1]))))
            # print('a:', list_activation[i].shape)
#     list_z.append(np.dot(list_activation[-1], np.transpose(list_nn_params[-1])))
    list_z.append(np.matmul(list_activation[-1], list_nn_params[-1].T))
    list_activation.append(sigmoid(list_z[-1]))
    # print('Theta ultimo:', list_nn_params[-1].shape)
    # print('z ultimo:', list_z[-1].shape)
    # print('a ultimo:', list_activation[-1].shape)
    
    y_vec = np.tile(np.arange(0, architecture_layers_size[-1]), (m, 1)) == np.tile(y, (1, architecture_layers_size[-1]))
    # print(y_vec)
    cost = - (y_vec * np.log(list_activation[-1])) - ((1 - y_vec) * np.log(1 - list_activation[-1]))
    J = np.dot((1 / m), np.sum(cost))
    
    # regularization
    list_theta_excluding_bias = []
    for i in range(len(list_nn_params)):
#         list_theta_excluding_bias.insert(i, np.copy(list_nn_params[i][:, 1:]))
        list_theta_excluding_bias.insert(i, list_nn_params[i][:, 1:].copy())
    sum_theta_excluding_bias = 0
    for i in range(len(list_theta_excluding_bias)):
        sum_theta_excluding_bias = sum_theta_excluding_bias + np.sum(list_theta_excluding_bias[i]**2)
    J = J + (lam / (2 * m)) * sum_theta_excluding_bias
    
    # Back propagation
    list_delta = []
    for i in range(len(list_nn_params)):
        list_delta.insert(i, np.zeros((list_nn_params[i].shape)))
        # print(list_delta[i].shape)
    # print(len(list_delta))
    num_z = len(list_z)
    for example in range(m):
        list_activation_i = []
        list_d = []
        list_z_i = []
        for a in range(len(list_activation)):
            list_activation_i.insert(a, list_activation[a][example, :].copy())
            # print('a iesimo:', list_activation_i[a])
        y_vec_i = y_vec[example, :].copy()
        # print(y_vec_i)
        for z in range(num_z - 1):
            list_z_i.insert(z, list_z[z][example, :].copy())
            # print('z iesimo:', list_z_i[0])
        list_d.append(list_activation_i[-1] - y_vec_i)
        list_d.insert(-1, np.matmul(list_nn_params[-1].T, list_d[-1]) * sigmoid_gradient(np.hstack((1, list_z_i[-1]))))
        if len(architecture_layers_size) > 3:
            for k in range(num_z - 2):
                list_d.insert(0, np.matmul(list_nn_params[num_z - 2 - k].T, list_d[0][1:]) * sigmoid_gradient(np.hstack((1, list_z_i[num_z - 3 - k]))))
        if len(architecture_layers_size) > 2:
            for i in range(len(list_delta) - 1):
                # print('delta_i:', list_delta[i].shape)
                list_delta[i] = list_delta[i] + np.matmul(list_d[i][1:][:, np.newaxis], list_activation_i[i][:, np.newaxis].T)
                # print('d_i:', list_d[i][1:][:, np.newaxis].shape)
                # print('a_i:', list_activation_i[i][:, np.newaxis].T.shape)
        list_delta[-1] = list_delta[-1] + np.matmul(list_d[-1][:, np.newaxis], list_activation_i[-2][:, np.newaxis].T)
    
    # regularization
    list_theta_zeroed_bias = []
    list_theta_grad = []
    for i in range(len(list_nn_params)):
        list_theta_zeroed_bias.insert(i, np.hstack((np.zeros((list_nn_params[i].shape[0], 1)), list_theta_excluding_bias[i])))
    for i in range(len(list_nn_params)):
        list_theta_grad.insert(i, ((1 / m) * list_delta[i] + (lam / m) * list_theta_zeroed_bias[i]))
    
    #unroll gradients
    nn_params = list_theta_grad[0].flatten()
    for i in range(1, len(list_theta_grad)):
        nn_params = np.hstack((nn_params, list_theta_grad[i].flatten()))
    
    return J, nn_params

# Import data from text file and NN train

In [ ]:
csvdata = np.matrix(pd.read_csv("iris.csv", header=None))
print('Number of examples:', csvdata.shape[0])
num_labels = 3
c = csvdata.shape[1]
X_complete = np.array(csvdata[:, 0:(c - 1)])
y_complete = np.array(csvdata[:, (c - 1)])

# print(X_complete[0])
# print(y_complete[0])

# Generate Training, CV and Test sets
set_size = y_complete.size
rand_indices = np.random.permutation(set_size)

X_complete = X_complete[rand_indices, :]
y_complete = y_complete[rand_indices]

train_size = np.ceil(set_size * 0.6).astype('uint8')
cv_size = np.ceil(set_size * 0.2).astype('uint8')

# X = np.copy(X_complete[0:train_size, :])
# y = np.copy(y_complete[0:train_size])
X = X_complete[0:train_size, :].copy()
y = y_complete[0:train_size].copy()

# X_cv = np.copy(X_complete[(train_size) : (train_size + cv_size), :])
# y_cv = np.copy(y_complete[(train_size) : (train_size + cv_size)])
X_cv = X_complete[(train_size) : (train_size + cv_size), :].copy()
y_cv = y_complete[(train_size) : (train_size + cv_size)].copy()

# X_test = np.copy(X_complete[(train_size + cv_size):, :])
# y_test = np.copy(y_complete[(train_size + cv_size):])
X_test = X_complete[(train_size + cv_size):, :].copy()
y_test = y_complete[(train_size + cv_size):].copy()

# Feautures normalization
X, mu, sigma = featureNormalize(X)
X_cv = np.divide(np.subtract(X_cv, mu), sigma)
X_test = np.divide(np.subtract(X_test, mu), sigma)

input_layer_size = X.shape[1]
# print(input_layer_size)
list_hidden_layers = [5, 6, 3]
# list_hidden_layers = [4]
# list_hidden_layers = [5, 2, 4, 3]
lam = 0
iterations = 10
architecture_layers_size = [input_layer_size, num_labels]
for i in range(len(list_hidden_layers)):
    architecture_layers_size.insert(i + 1, list_hidden_layers[i])

list_initial_theta = rand_initialize_weights(architecture_layers_size)

theta = list_initial_theta[0].flatten()
for i in range(1, len(list_initial_theta)):
    theta = np.hstack((theta, list_initial_theta[i].flatten()))
    
[J, theta] = fmin_cg(cost_function, x0 = theta, fprime = None, args = (X, y, architecture_layers_size, lam), maxiter = iterations)